# 1. Install Dependencies and Setup

In [ ]:
# just making sure Python is happy
print("Hello, world!")

In [ ]:
import tensorflow as tf
import os
import cv2
import numpy as np
from numba import cuda

from matplotlib import pyplot as plt
from keras import backend as k
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# set hyperprameters and top-level variables
data_dir = 'data3'
log_dir = 'logs'
model_dir = 'models'
best_weights = os.path.join(model_dir,'weights.best.hdf5')

imgsize = 750
batchsize = 32
epochs = 32
learning_rate = 0.001           # default 0.001
epsilon = 1e-07                 # default 1e-07
dropout_rate = 0.25

In [ ]:
# function to determine rough model memory requirements
def get_model_memory_usage(batch_size, model_to_review):
    shapes_mem_count = 0
    internal_model_mem_count = 0
    for l in model_to_review.layers:
        layer_type = l.__class__.__name__
        if layer_type == 'Model':
            internal_model_mem_count += get_model_memory_usage(batch_size, l)
        single_layer_mem = 1
        out_shape = l.output_shape
        if type(out_shape) is list:
            out_shape = out_shape[0]
        for s in out_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([k.count_params(p) for p in model_to_review.trainable_weights])
    non_trainable_count = np.sum([k.count_params(p) for p in model_to_review.non_trainable_weights])

    number_size = 4.0
    if k.floatx() == 'float16':
        number_size = 2.0
    if k.floatx() == 'float64':
        number_size = 8.0

    total_memory = number_size * (batch_size * shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3) + internal_model_mem_count
    return gbytes

In [ ]:
# # Hide the GPU - forces CPU training
# tf.config.set_visible_devices([], 'GPU')

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
   tf.config.experimental.set_memory_growth(gpu, True)
   #tf.config.experimental.set_virtual_device_configuration(gpu, [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=8192)])
tf.config.list_physical_devices('GPU')

# 2. Load, Scale, and Review Data

In [ ]:
# load and scale the data
data = tf.keras.utils.image_dataset_from_directory(data_dir, batch_size=batchsize, image_size=(imgsize, imgsize))
data = data.map(lambda x,y: (x/255, y))

In [ ]:
# review the data parameters
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()
print(batch[0][0])
print(batch[1])
print(batch[0].shape)

In [ ]:
# review the data as images
fig, ax = plt.subplots(ncols=5, figsize=(20, 20))
for idx, img in enumerate(batch[0][:5]):
    ax[idx].imshow(img.astype(int))
    # ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

# 3. Split Data

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)
print("Data size: ", len(data))
print("Train size: ", train_size)
print("Val size: ", val_size)
print("Test size: ", test_size)
print("Total size: ", train_size+val_size+test_size)

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# 4. Build Deep Learning Model

In [ ]:
# create a sequential model
model = Sequential()

# convolution layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(imgsize,imgsize,3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# flatten layer
model.add(Flatten())

# dense layers with dropout
model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(64, activation='relu'))
model.add(Dropout(dropout_rate))

# output layer
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# set up an optimizer and compile
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon)
model.compile(optimizer=optimizer, loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
# define callbacks
tensor_board = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint(best_weights, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_accuracy', verbose=1, patience=5, restore_best_weights=True)
callbacks_list = [tensor_board, checkpoint, early_stopping]

In [ ]:
# how much memory might we need - limit around 6GB???
get_model_memory_usage(batchsize, model)

In [ ]:
# summarize the model
model.summary()

# 5. Train

In [ ]:
hist = model.fit(train, epochs=epochs, validation_data=val, callbacks=callbacks_list)

# 6. Plot Performance

In [ ]:
# plot loss
f, ax = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(7, 9))
ax[0].plot([None] + hist.history['loss'], 'o-')
ax[0].plot([None] + hist.history['val_loss'], 'x-')
ax[0].legend(['Train loss', 'Validation loss'], loc = 0)
ax[0].set_title('Training/Validation Loss per Epoch')
ax[0].set_ylabel('Loss')

ax[1].plot([None] + hist.history['accuracy'], 'o-')
ax[1].plot([None] + hist.history['val_accuracy'], 'x-')
ax[1].legend(['Train Accuracy', 'Validation Accuracy'], loc = 0)
ax[1].set_title('Training/Validation Accuracy per Epoch')
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Accuracy')

# 7. Evaluate

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print("-----------------------------------------------------")
print("Precision:       ", pre.result())
print("Recall:          ", re.result())
print("Binary Accuracy: ", acc.result())
print("-----------------------------------------------------")

# 8. Validate and Test

In [ ]:
img01 = cv2.imread(data_dir + '/_000/_0-01.jpg')
img02 = cv2.imread(data_dir + '/_000/_0-02.jpg')
img03 = cv2.imread(data_dir + '/_000/_0-03.jpg')
img04 = cv2.imread(data_dir + '/_000/_0-04.jpg')
img05 = cv2.imread(data_dir + '/_000/_0-05.jpg')

img11 = cv2.imread(data_dir + '/_FAPT/_F-01.jpg')
img12 = cv2.imread(data_dir + '/_FAPT/_F-02.jpg')
img13 = cv2.imread(data_dir + '/_FAPT/_F-03.jpg')
img14 = cv2.imread(data_dir + '/_FAPT/_F-04.jpg')
img15 = cv2.imread(data_dir + '/_FAPT/_F-05.jpg')

imgT01 = cv2.imread('test/_T-01.jpg')
imgT02 = cv2.imread('test/_T-02.jpg')
imgT13 = cv2.imread('test/_T-03.jpg')
imgT14 = cv2.imread('test/_T-04.jpg')
imgT15 = cv2.imread('test/_T-05.jpg')

In [ ]:
print(img01.shape)
# plt.imshow(cv2.cvtColor(img01, cv2.COLOR_BGR2RGB))
# plt.show()

In [ ]:
resize01 = tf.image.resize(img01, (imgsize,imgsize))
resize02 = tf.image.resize(img02, (imgsize,imgsize))
resize03 = tf.image.resize(img03, (imgsize,imgsize))
resize04 = tf.image.resize(img04, (imgsize,imgsize))
resize05 = tf.image.resize(img05, (imgsize,imgsize))

resize11 = tf.image.resize(img11, (imgsize,imgsize))
resize12 = tf.image.resize(img12, (imgsize,imgsize))
resize13 = tf.image.resize(img13, (imgsize,imgsize))
resize14 = tf.image.resize(img14, (imgsize,imgsize))
resize15 = tf.image.resize(img15, (imgsize,imgsize))

resizeT01 = tf.image.resize(imgT01, (imgsize,imgsize))
resizeT02 = tf.image.resize(imgT02, (imgsize,imgsize))
resizeT13 = tf.image.resize(imgT13, (imgsize,imgsize))
resizeT14 = tf.image.resize(imgT14, (imgsize,imgsize))
resizeT15 = tf.image.resize(imgT15, (imgsize,imgsize))

In [ ]:
print(resize01.shape)
# plt.imshow(cv2.cvtColor(resize01.numpy().astype(np.uint16), cv2.COLOR_BGR2RGB))
# plt.show()

In [ ]:
yhat01 = model.predict(np.expand_dims(resize01/255, 0))[0][0]
yhat02 = model.predict(np.expand_dims(resize02/255, 0))[0][0]
yhat03 = model.predict(np.expand_dims(resize03/255, 0))[0][0]
yhat04 = model.predict(np.expand_dims(resize04/255, 0))[0][0]
yhat05 = model.predict(np.expand_dims(resize05/255, 0))[0][0]

yhat11 = model.predict(np.expand_dims(resize11/255, 0))[0][0]
yhat12 = model.predict(np.expand_dims(resize12/255, 0))[0][0]
yhat13 = model.predict(np.expand_dims(resize13/255, 0))[0][0]
yhat14 = model.predict(np.expand_dims(resize14/255, 0))[0][0]
yhat15 = model.predict(np.expand_dims(resize15/255, 0))[0][0]

yhatT01 = model.predict(np.expand_dims(resizeT01/255, 0))[0][0]
yhatT02 = model.predict(np.expand_dims(resizeT02/255, 0))[0][0]
yhatT13 = model.predict(np.expand_dims(resizeT13/255, 0))[0][0]
yhatT14 = model.predict(np.expand_dims(resizeT14/255, 0))[0][0]
yhatT15 = model.predict(np.expand_dims(resizeT15/255, 0))[0][0]

In [ ]:
print("{:.11f}".format(yhat01), " - Verify 0.0")
print("{:.11f}".format(yhat02), " - Verify 0.0")
print("{:.11f}".format(yhat03), " - Verify 0.0")
print("{:.11f}".format(yhat04), " - Verify 0.0")
print("{:.11f}".format(yhat05), " - Verify 0.0")
print("------------------------")
print("{:.11f}".format(yhat11), " - Verify 1.0")
print("{:.11f}".format(yhat12), " - Verify 1.0")
print("{:.11f}".format(yhat13), " - Verify 1.0")
print("{:.11f}".format(yhat14), " - Verify 1.0")
print("{:.11f}".format(yhat15), " - Verify 1.0")
print("------------------------")
print("{:.11f}".format(yhatT01), " - Want 0.0")
print("{:.11f}".format(yhatT02), " - Want 0.0")
print("{:.11f}".format(yhatT13), " - Want 1.0")
print("{:.11f}".format(yhatT14), " - Want 1.0")
print("{:.11f}".format(yhatT15), " - Want 1.0")

# 9. Save the Model

In [ ]:
# model.save(os.path.join('models','imageclassifier.h5'))
# new_model = load_model(os.path.join('models', 'imageclassifier.h5'))
# yhatnew = new_model.predict(np.expand_dims(resize01/255, 0))
# print(yhatnew)

# 10. Release Memory

In [ ]:
# k.clear_session()

In [ ]:
# cuda.select_device(0)
# cuda.close()